# Managing your data

By running our own simulations and workflows, we have now created a bunch of data along the way. So let's see how we can
explore it, shall we?

As mentioned earlier, AiiDA stores it's data in an internal file repository, as well as an SQL database. The former
contains the main input and output files of the calculations, while the latter contains the provenance data, as well as
parsed results (only pointers to files are stored in the SQL database). To achieve high performance, these storage
solutions are machine-readable, rather than human-readable, so we will require the AiiDA CLI and API to efficiently
explore our data. Let's first load our AiiDA profile again:

In [ ]:
%load_ext aiida
%aiida

## Dumping of process data to disk

As of AiiDA v2.6.1, released on 2024-07-01, it is now possible to dump all data involved in the execution of a
calculation or workflow to disk in a human-readable, logical directory structure. To achieve this, simply run the
following command, e.g. using the PK of your finished `EquationOfStateWorkChain`:

In [ ]:
%verdi process dump <PK>

Once the command ran successfully, you will now be able to see a directory with the name
`dump-EquationOfStateWorkChain-<PK>` in the file explorer of your jupyter instance. The contained files can now be
explored using the common Linux command-line tools we all know and love, such as `grep`.

More information on the dumping can be found in the [relevant documentation section](https://aiida.readthedocs.io/projects/aiida-core/en/stable/howto/data.html#dumping-data-to-disk).

## Grouping data

Talking of directories, it is possible (and recommended for larger studies) to organize your `Node` entities in a
logical structure, reminiscent of how one would organize their files and folders in a typical computational research
project.

For this, AiiDA provides the concept of `Group`s. Let's have a look at our current database using:

In [ ]:
%verdi group list -aA

We see that the pseudopotentials we downloadad previously are actually saved as `Group`s in our database. If we want to
create our own groups and add nodes to it, we can do so using:

In [ ]:
%verdi group create <our-group>
%verdi group add-nodes -G <our-group> <PK> <PK>

Where we can supply a list of PKs or UUIDs to be added to `<our-group>`. To create the previously mentioned
directory-like organization, we can use slashes in the group names to create sub-groups that behave like sub-folders
would on your file system.

This concludes our short introduction to `Group`s, and we point you to the [relevant documentation section for further
reading]().

## The QueryBuilder

The main class that can be used for querying data stored in the AiiDA database is the `QueryBuilder`. In this section we
will learn how you can use it to explore your previously created data. It transforms your queries written in Python to
SQL. 
So let's get started by importing and instantiating it:

In [ ]:
from aiida import orm

qb = orm.QueryBuilder()
qb

: 

We can now `append` entities and relations we want to be querying for. Let's first count how many nodes we actually have
in our database:

In [ ]:
qb.append(orm.Node)
qb.count()

In addition to our simulation results, as we installed the pseudopotentials for the whole periodic table, we already have quite
a few nodes contained in our database. Let's now see the`PwCalculation`s we have executed.
Note: As this will be an independent query, we will have to create another instance of the `QueryBuilder`:

In [ ]:
from aiida_quantumespresso.calculations.pw import PwCalculation

qb = orm.QueryBuilder()
qb.append(PwCalculation)
qb.all()

Nice, all of our calculations are there. As you could see, the query returned the full node of each `PwCalculation`. If
we would only like to obtain individual properties, e.g. the PK, we can `project` those:

In [ ]:
qb = orm.QueryBuilder()
qb.append(PwCalculation, project='pk')
qb.all()

In addition, we can also filter our results. Say, we would only like to obtain calculations that we ran today. We can do so with the following:

In [ ]:
from datetime import datetime, timedelta

qb = orm.QueryBuilder()
qb.append(
    PwCalculation,
    filters={'ctime': {'>': datetime.now() - timedelta(days=1)}}
)
qb.all()

Importantly, it's also possible to chain querying expressions together. This allows you to construct (almost)
arbitrarily complex queries. For example, let's obtain the output `StructureData` nodes that were created by our
`PwCalculation`s of today. These could be of interest, for instance, if we relaxed the crystal structure:

In [ ]:
qb = orm.QueryBuilder()
qb.append(
    PwCalculation,
    filters={'ctime': {'>': datetime.now() - timedelta(days=1)}},
    tag="calculation"
)
qb.append(orm.StructureData, with_incoming="calculation")
qb.all()

To achieve this, we first `tag` our query for the `PwCalculation`s, so that we can link it to the second query we append to the `QueryBuilder`. We use the `with_incoming` argument, as the `StructureData` is an outgoing link of the `PwCalculation`.

This concludes our short introduction to the `QueryBuilder`. Of course, it provides many more capabilities so we refer
you to the [relevant documentation on querying for data]().

We have now reached the end of our tutorial notebooks. We hope we could give you an overview of how you can get
started with AiiDA and what it can do for you.

If you want to continue your journey, good next steps are the [extensive AiiDA documentation on `readthedocs`](https://aiida.readthedocs.io/projects/aiida-core/en/stable/)
where you can find `How-To` guides on
common tasks, as well as in-depth information about the cogs and wheels that power AiiDA.

We are also holding **regular week-long (virtual) tutorials**. The content of the last one can be found [here](https://aiida-tutorials.readthedocs.io/en/latest/), and goes
much more in-depth than we could do so in the limited time today. We are also planning another virtual tutorial in early 2025,
which will also highlight recent new features of AiiDA and improvements on usability, so keep your eyes
open.

If you have any issues, don't hesitate to reach out for help on our [Discourse support platform](https://aiida.discourse.group). Lastly, if you
want to use and further explore AiiDA, but are not very keen on coding, [AiiDAlab](https://aiidalab.readthedocs.io/en/latest/) provides you with a jupyter-based
graphical user interface that you can use to run the turn-key workflows provided by AiiDA with just a few clicks.

Happy computing!

The AiiDA team